In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nhl-game-data-20132021/teamInfo.csv
/kaggle/input/nhl-game-data-20132021/playsStats.csv
/kaggle/input/nhl-game-data-20132021/gameStats.csv
/kaggle/input/nhl-game-data-20132021/basicGameStatsSeasonCumulative.csv
/kaggle/input/nhl-game-data-20132021/gamePlayerStats.csv
/kaggle/input/nhl-game-data-20132021/penalties.csv
/kaggle/input/nhl-game-data-20132021/gameGoalieStats.csv
/kaggle/input/nhl-game-data-20132021/gameInfo.csv
/kaggle/input/nhl-game-data-20132021/goals.csv
/kaggle/input/nhl-game-data-20132021/playPlayersStats.csv
/kaggle/input/nhl-game-data-20132021/playerInfo.csv
/kaggle/input/nhl-game-data-20132021/shifts.csv
/kaggle/input/nhl-game-data-20132021/gameOfficialsInfo.csv


In [2]:
import pandas as pd

teamInfoDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/teamInfo.csv')
playsStatsDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/playsStats.csv')
gameStatsDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/gameStats.csv')
gamePlayerStatsDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/gamePlayerStats.csv')
penaltiesDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/penalties.csv')
gameGoalieStatsDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/gameGoalieStats.csv')
gameInfoDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/gameInfo.csv')
goalsDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/goals.csv')
playPlayersStatsDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/playPlayersStats.csv')
playerInfoDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/playerInfo.csv')
shiftsDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/shifts.csv')
gameOfficialsInfoDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/gameOfficialsInfo.csv')
basicGameStatsSeasonCumulativeDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/basicGameStatsSeasonCumulative.csv')



In [3]:
def getTeam(id):
    '''
    Helper function to make storing running team stats easier
    Stored id -> array id
    '''
    # Handle Phoenix -> Arizona
    if id == 27:
        return 29
        
    elif id < 12:
        return id - 1
    
    elif id < 28:
        return id - 2
    
    elif id <= 30:
        return id - 3
    
    else:
        return id - 24
    

def getTeamId(id):
    '''
    Helper function to make storing running team stats easier
    Array id -> stored id
    '''
    # Handle Phoenix -> Arizona
    if id == 29:
        return 27
    
    if id < 11:
        return id + 1
    
    elif id < 26:
        return id + 2
    
    elif id <= 27:
        return id + 3
    
    else:
        return id + 24


In [6]:
# GF,GA,GD,PP,PK,SP,SvP,WS,CS,FC,PDO,55GA
allUnblockedShotsDf = playsStatsDf[(playsStatsDf['event'] == 'Shot') | (playsStatsDf['event'] == 'Goal') | (playsStatsDf['event'] == 'Missed Shot')]

seasons = [season for season in range(2013, 2022)]
gameDataPerTeam = []

for season in seasons:
    # Reset season cumulative stats
    GF = [0 for _ in range(32)]
    GA = [0 for _ in range(32)]
    GD = [0 for _ in range(32)]
    PP = [50 for _ in range(32)]
    PK = [50 for _ in range(32)]
    SP = [50 for _ in range(32)]
    SvP = [50 for _ in range(32)]
    WS = [0 for _ in range(32)]
    FC = [50 for _ in range(32)]
    PDO = [100 for _ in range(32)]
    GA55 = [1 for _ in range(32)]

    powerplays = [0 for _ in range(32)]
    ppGoals = [0 for _ in range(32)]
    penaltyKills = [0 for _ in range(32)]
    penaltyKillGoalsAgainst = [0 for _ in range(32)]
    shots = [0 for _ in range(32)]
    saves = [0 for _ in range(32)]
    shotsFaced = [0 for _ in range(32)]

    unblockedShotsClose = [[0,0] for _ in range(32)] #[for, against]

    #print(games)
    
    seasonGames = gameInfoDf[gameInfoDf['ID'].astype(str).str.startswith(str(season))]
    games = seasonGames['ID'].unique()
    print(seasonGames)
    
    for game in games:
        gameInfo = gameInfoDf.loc[gameInfoDf['ID'] == game]
        #print(gameInfo)
        #i = gameInfo.iloc[0]['homeTeam']
        #print(i)
        home = getTeam(gameInfo.iloc[0]['homeTeam'])
        away = getTeam(gameInfo.iloc[0]['awayTeam'])


        # Create examples (stats taken before game played)
        if gameInfo.iloc[0]['homeTeam'] == 27:
            homeTeam = 'Arizona Coyotes'
        else:
            homeTeam = teamInfoDf[teamInfoDf['ID'] == gameInfo.iloc[0]['homeTeam']]
            homeTeam = homeTeam.iloc[0]['Name']
        #print(gameInfo.iloc[0]['homeTeam'])
        #print(homeTeam)
        #print(homeTeam.iloc[0]['Name'])
        homeData = [game, homeTeam, 'Home', FC[home], GF[home], GA[home], GD[home], PP[home], PK[home], SP[home], SvP[home], PDO[home], WS[home], GA55[home]]

        if gameInfo.iloc[0]['awayTeam'] == 27:
            awayTeam = 'Arizona Coyotes'
        else:
            awayTeam = teamInfoDf[teamInfoDf['ID'] == gameInfo.iloc[0]['awayTeam']]
            awayTeam = awayTeam.iloc[0]['Name']

        awayData = [game, awayTeam, 'Away', FC[away], GF[away], GA[away], GD[away], PP[away], PK[away], SP[away], SvP[away], PDO[away], WS[away], GA55[away]]



        # GF
        GF[home] += gameInfo.iloc[0]['homeGoals']
        GF[away] += gameInfo.iloc[0]['awayGoals']

        # GA
        GA[home] += gameInfo.iloc[0]['awayGoals']
        GA[away] += gameInfo.iloc[0]['homeGoals']

        # GD
        GD[home] = GF[home] - GA[home]
        GD[away] = GF[away] - GA[away]


        gameStats = gameStatsDf.loc[gameStatsDf['gameId'] == game]

        # PP, PK
        powerplays[home] += gameStats.iloc[0]['homePowerPlayOpportunities']
        powerplays[away] += gameStats.iloc[0]['awayPowerPlayOpportunities']

        ppGoals[home] += gameStats.iloc[0]['homePowerPlayGoals']
        ppGoals[away] += gameStats.iloc[0]['awayPowerPlayGoals']

        penaltyKills[home] += gameStats.iloc[0]['awayPowerPlayOpportunities']
        penaltyKills[away] += gameStats.iloc[0]['homePowerPlayOpportunities']

        penaltyKillGoalsAgainst[home] += gameStats.iloc[0]['awayPowerPlayGoals']
        penaltyKillGoalsAgainst[away] += gameStats.iloc[0]['homePowerPlayGoals']

        if powerplays[home] != 0:
            PP[home] = (ppGoals[home] / powerplays[home]) * 100
            PK[away] = (1 - (penaltyKillGoalsAgainst[away] / penaltyKills[away])) * 100

        if powerplays[away] != 0:
            PP[away] = (ppGoals[away] / powerplays[away]) * 100
            PK[home] = (1 - (penaltyKillGoalsAgainst[home] / penaltyKills[home])) * 100


        # SP
        shots[home] += gameStats.iloc[0]['homeShots']
        shots[away] += gameStats.iloc[0]['awayShots']

        SP[home] = (GF[home] / shots[home]) * 100
        SP[away] = (GF[away] / shots[away]) * 100


        # SvP
        saves[home] += gameStats.iloc[0]['awayShots'] - gameInfo.iloc[0]['awayGoals']
        saves[away] += gameStats.iloc[0]['homeShots'] - gameInfo.iloc[0]['homeGoals']

        shotsFaced[home] += gameStats.iloc[0]['awayShots']
        shotsFaced[away] += gameStats.iloc[0]['homeShots']

        SvP[home] = (saves[home] / shotsFaced[home]) * 100
        SvP[away] = (saves[away] / shotsFaced[away]) * 100


        # WS
        homeLabel = ""
        awayLabel = ""
        if "win" in gameInfo.iloc[0]['result']:
            WS[home] += 1
            WS[away] = 0
            homeLabel = "Win"
            awayLabel = "Loss"

        else:
            WS[away] += 1
            WS[home] = 0
            homeLabel = "Loss"
            awayLabel = "Win"


        #FC
        gameUnblockedShotsDf = allUnblockedShotsDf[allUnblockedShotsDf['playId'].str.startswith(str(game))] 
        #print(allUnblockedShotsDf)
        #print(gameUnblockedShotsDf)
        for playId in gameUnblockedShotsDf['playId'].unique():
            #print(playId)
            play = gameUnblockedShotsDf.loc[gameUnblockedShotsDf['playId'] == playId]
            scoreDiff = abs(play.iloc[0]['homeGoals'] - play.iloc[0]['awayGoals'])

            #print(scoreDiff)
            if play.iloc[0]['period'] <= 2:
                # Close game: scoreDiff <= 1
                if scoreDiff <= 1:
                    #print(play.iloc[0]['teamIdFor'])
                    # Add for shot
                    unblockedShotsClose[getTeam(int(play.iloc[0]['teamIdFor']))][0] += 1 
                    # Add against shot
                    unblockedShotsClose[getTeam(int(play.iloc[0]['teamIdAgainst']))][1] += 1 

            elif play.iloc[0]['period'] == 3:
                # Close game: tied game
                if scoreDiff == 0:
                    # Add for shot
                    unblockedShotsClose[getTeam(int(play.iloc[0]['teamIdFor']))][0] += 1
                    # Add against shot
                    unblockedShotsClose[getTeam(int(play.iloc[0]['teamIdAgainst']))][1] += 1 

        #print(unblockedShotsClose)          
        if unblockedShotsClose[home] != (0,0):
            FC[home] = (unblockedShotsClose[home][0] / (unblockedShotsClose[home][0] + unblockedShotsClose[home][1])) * 100

        if unblockedShotsClose[away] != (0,0):
            FC[away] = (unblockedShotsClose[away][0] / (unblockedShotsClose[away][0] + unblockedShotsClose[away][1])) * 100




        #PDO
        PDO[home] = SP[home] + SvP[home]
        PDO[away] = SP[away] + SvP[away]


        # GA55
        if GA[home] != penaltyKillGoalsAgainst[home]:
            GA55[home] = (GF[home] - ppGoals[home]) / (GA[home] - penaltyKillGoalsAgainst[home])

        if GA[away] != penaltyKillGoalsAgainst[away]:
            GA55[away] = (GF[away] - ppGoals[away]) / (GA[away] - penaltyKillGoalsAgainst[away])



        # Add labels to examples and append to all games
        homeData.append(homeLabel)
        awayData.append(awayLabel)

        gameDataPerTeam.append(homeData)
        gameDataPerTeam.append(awayData)
    


df = pd.DataFrame(data=gameDataPerTeam, columns=['GameID', 'Team', 'Location', 'Fenwick Close %', 'GF', 'GA', 'GD', 'PP%', 'PK%', 'Sh%', 'Sv%', 'PDO', 'Win Streak', '5-5 F/A', 'Label'])
df.to_csv('basicGameStatsSeasonCumulative.csv', index=False)
print(df)
    
    
    

              ID                  date  homeTeam  awayTeam  homeGoals  \
0     2013020001  2013-10-01T23:00:00Z         8        10          3   
1     2013020002  2013-10-02T00:00:00Z        16        15          6   
2     2013020003  2013-10-02T02:00:00Z        22        52          4   
3     2013020004  2013-10-02T23:30:00Z         4        10          1   
4     2013020005  2013-10-03T00:00:00Z        17         7          2   
...          ...                   ...       ...       ...        ...   
1225  2013021226  2014-04-13T23:30:00Z         5         9          2   
1226  2013021227  2014-04-14T00:00:00Z        30        18          3   
1227  2013021228  2014-04-14T01:00:00Z        23        20          5   
1228  2013021229  2014-04-14T00:00:00Z        24        21          3   
1229  2013021230  2014-04-14T01:00:00Z        27        25          2   

      awayGoals    result homeRinkSide  
0             4  REG loss        right  
1             4   REG win        right  
